# Scikit-Learn Cross Val Score on Logistic Regression Model
Using IRIS_VIEW from DWC. This view has 150 records

## Install fedml_aws library

In [1]:
pip install fedml_aws-1.0.0-py3-none-any.whl --force-reinstall

Processing ./fedml_aws-1.0.0-py3-none-any.whl
  Using cached hdbcli-2.10.13-cp34-abi3-manylinux1_x86_64.whl (11.7 MB)
  Attempting uninstall: hdbcli
    Found existing installation: hdbcli 2.10.13
    Uninstalling hdbcli-2.10.13:
      Successfully uninstalled hdbcli-2.10.13
  Attempting uninstall: fedml-aws
    Found existing installation: fedml-aws 1.0.0
    Uninstalling fedml-aws-1.0.0:
      Successfully uninstalled fedml-aws-1.0.0
Note: you may need to restart the kernel to use updated packages.


## Import Libraries

In [2]:
from fedml_aws import DwcSagemaker
from fedml_aws import DbConnection
import numpy as np
import pandas as pd

## Create DwcSagemaker instance to access libraries functions

In [3]:
dwcs = DwcSagemaker(prefix='scikit-learn/cross-val-iris', bucket_name='fedml-bucket')

## Create DbConnection instance to get data from DWC

Before running the following cell, you should have a config.json file in the same directory as this notebook with the specified values to allow you to access to DWC.

You should also have the follow view `IRIS_VIEW` created in your DWC. To gather this data, please refer to https://www.kaggle.com/uciml/iris

In [4]:
%%time
db = DbConnection()
train_data = db.get_data_with_headers(table_name='IRIS_VIEW', size=1)
data = pd.DataFrame(train_data[0], columns=train_data[1])
data

CPU times: user 44.8 ms, sys: 3.49 ms, total: 48.3 ms
Wall time: 108 ms


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3,5.2,2.3,Iris-virginica
146,6.3,2.5,5,1.9,Iris-virginica
147,6.5,3,5.2,2,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
data.isna().any()

sepal_length    False
sepal_width     False
petal_length    False
petal_width     False
species         False
dtype: bool

In [6]:
data.isnull().any()

sepal_length    False
sepal_width     False
petal_length    False
petal_width     False
species         False
dtype: bool

In [7]:
data.dtypes

sepal_length    object
sepal_width     object
petal_length    object
petal_width     object
species         object
dtype: object

In [8]:
from sklearn.model_selection import train_test_split
data = data.sample(frac=1).reset_index(drop=True)
sub_data = data.head(100)
X_train, X_test, y_train, y_test = train_test_split(sub_data.drop(['species'], axis=1), sub_data['species'], test_size=0.3)

In [9]:
train_data = pd.concat([X_train, y_train], axis=1)
train_data

,sepal_length,sepal_width,petal_length,petal_width,species
68,6.1,2.6,5.6,1.4,Iris-virginica
73,5,3.6,1.4,0.2,Iris-setosa
15,4.9,2.5,4.5,1.7,Iris-virginica
39,4.9,3.1,1.5,0.1,Iris-setosa
10,5.4,3.4,1.5,0.4,Iris-setosa
...,...,...,...,...,...
53,6.5,3,5.5,1.8,Iris-virginica
58,4.3,3,1.1,0.1,Iris-setosa
85,5.3,3.7,1.5,0.2,Iris-setosa
59,5.6,2.8,4.9,2,Iris-virginica


In [10]:
pd.Series(train_data['species']).unique()

array(['Iris-virginica', 'Iris-setosa', 'Iris-versicolor'], dtype=object)

In [11]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data

,sepal_length,sepal_width,petal_length,petal_width,species
44,6.7,3.1,4.4,1.4,Iris-versicolor
1,5.2,4.1,1.5,0.1,Iris-setosa
70,4.6,3.2,1.4,0.2,Iris-setosa
86,6.2,2.8,4.8,1.8,Iris-virginica
18,4.6,3.6,1,0.2,Iris-setosa
49,4.4,3,1.3,0.2,Iris-setosa
54,5.9,3,4.2,1.5,Iris-versicolor
11,7.7,3.8,6.7,2.2,Iris-virginica
51,6.9,3.1,4.9,1.5,Iris-versicolor
97,5,3.2,1.2,0.2,Iris-setosa


## Train SciKit Model

`train_data` is the data you want to train your model with.

In order to deploy a model to AWS using the Scikit-learn Sagemaker SDK, you must have a script that tells Sagemaker how to train and deploy the model. The path to the script is passed to the `train_sklearn_model` function in the `train_script` parameter.

`instance_type` specifies how much computing power we want AWS to allocate for our services.

In [12]:
clf = dwcs.train_sklearn_model(train_data=train_data,
                               test_data=test_data,
                               content_type='text/csv',
                               train_script='iris_trainV3.py',
                               instance_count=1,
                               instance_type='ml.c4.xlarge',
                               wait=True,
                               base_job_name='scikit-learn-logistic-regression-crossval'
                              )

Training data uploaded
Test data uploaded
2021-10-06 22:32:30 Starting - Starting the training job...
2021-10-06 22:32:54 Starting - Launching requested ML instancesProfilerReport-1633559549: InProgress
......
2021-10-06 22:33:54 Starting - Preparing the instances for training.........
2021-10-06 22:35:29 Downloading - Downloading input data......
2021-10-06 22:36:25 Training - Training image download completed. Training in progress..2021-10-06 22:36:26,902 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-10-06 22:36:26,904 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-10-06 22:36:26,915 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-10-06 22:36:34,479 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-10-06 22:36:34,490 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-10-06 22:36:34,503 sagema